# OSA Analysis Tool - Testing Notebook

This notebook allows you to test the OSA (On-Shelf Availability) analysis functions independently of the Streamlit frontend.

## Features Available for Testing:
- OSA API data fetching
- Image analysis with Google Gemini
- Accuracy metrics calculation
- SKU detection and validation
- Display image processing

## Setup
Make sure your `.env` file contains the required `GOOGLE_API_KEY` for Gemini API access.


In [1]:
# Import necessary libraries and modules
import sys
import os
import json
from datetime import datetime, timedelta
from PIL import Image
import pandas as pd
from io import BytesIO
import requests

# Import from our local modules
from config import (
    APP_TITLE, 
    APP_DESCRIPTION, 
    COLORS, 
    MAX_FILE_SIZE_MB,
    ALLOWED_EXTENSIONS,
    GOOGLE_API_KEY,
    GEMINI_MODEL
)

from gemini_client import GeminiProductDetector, create_sample_detection_result

# Import functions from app.py
from app import (
    fetch_all_osa_data,
    get_unique_display_ids,
    get_unique_dates,
    fetch_osa_images,
    get_unique_skus_from_api_data,
    get_ground_truth_skus,
    get_predicted_skus_from_results,
    get_sku_image_url,
    calculate_accuracy_metrics,
    validate_uploaded_file
)

print("✅ All imports successful!")
print(f"📊 App Title: {APP_TITLE}")
print(f"🤖 Gemini Model: {GEMINI_MODEL}")
print(f"🔑 API Key configured: {'Yes' if GOOGLE_API_KEY else 'No'}")


✅ All imports successful!
📊 App Title: OSA Image Analysis Tool
🤖 Gemini Model: gemini-2.5-pro
🔑 API Key configured: Yes


## 1. Testing OSA API Data Fetching

First, let's test the OSA API functions to fetch data and explore available display IDs and dates.


In [2]:
# Test fetching all OSA data
print("🔍 Fetching all OSA data...")
all_data, error = fetch_all_osa_data()

if error:
    print(f"❌ Error fetching data: {error}")
else:
    print(f"✅ Successfully fetched {len(all_data)} records")
    
    # Get unique display IDs and dates
    display_ids = get_unique_display_ids(all_data)
    dates = get_unique_dates(all_data)
    
    print(f"📍 Available Display IDs: {len(display_ids)}")
    print(f"   {display_ids[:5]}{'...' if len(display_ids) > 5 else ''}")
    
    print(f"📅 Available Dates: {len(dates)}")
    print(f"   Latest: {dates[0]['display'] if dates else 'None'}")
    print(f"   Oldest: {dates[-1]['display'] if dates else 'None'}")
    
    # Show sample record structure
    if all_data:
        print("\n📋 Sample record structure:")
        sample_keys = list(all_data[0].keys())
        print(f"   Keys: {sample_keys}")


🔍 Fetching all OSA data...
✅ Successfully fetched 14799 records
📍 Available Display IDs: 40
   ['ACH136', 'ACH137', 'ACH138', 'ACH139', 'ACH140']...
📅 Available Dates: 10
   Latest: 2025-09-11 (Thursday)
   Oldest: 2025-09-02 (Tuesday)

📋 Sample record structure:
   Keys: ['DOEntry', 'DisplayID', 'ArticleNo', 'Upccode', 'SKU', 'OSA', 'ShelfNo', 'FacingPOG', 'FacingTouching', 'AfterImagePath', 'SKUImage']


## 2. Testing Specific Display Data Fetching

Now let's fetch data for a specific date and display ID to analyze images and SKUs.


In [3]:
# Select a date and display ID for testing
# You can modify these values based on the available data from the previous cell
test_date = "2025-09-08"  # Change this to a date from your available dates
test_display_id = "ACH194"  # Change this to a display ID from your available IDs


print(f"🔍 Fetching OSA images for {test_date} - {test_display_id}")
osa_data, error = fetch_osa_images(test_date, test_display_id)
print(f"https://tamimi.impulseglobal.net//ExternalServices/IR_API.asmx/getOSAImage?Date={test_date}&DisplayID={test_display_id}")

if error:
    print(f"❌ Error fetching specific data: {error}")
    # Try with first available date and display ID if available
    if 'display_ids' in locals() and 'dates' in locals() and display_ids and dates:
        test_date = dates[0]['date']
        test_display_id = display_ids[0]
        print(f"🔄 Retrying with {test_date} - {test_display_id}")
        osa_data, error = fetch_osa_images(test_date, test_display_id)

if not error and osa_data:
    print(f"✅ Successfully fetched {len(osa_data)} items")
    
    # Analyze SKUs
    all_skus = get_unique_skus_from_api_data(osa_data)
    ground_truth_skus = get_ground_truth_skus(osa_data)
    
    print(f"\n📦 SKU Analysis:")
    print(f"   Total SKUs: {len(all_skus)}")
    print(f"   Ground Truth SKUs (OSA=1): {len(ground_truth_skus)}")
    
    # Display first few SKUs
    if all_skus:
        print(f"\n📋 Sample SKUs:")
        for i, sku in enumerate(all_skus[:5]):
            osa_status = "✅" if sku in ground_truth_skus else "❌"
            print(f"   {i+1}. {sku} {osa_status}")
    
    # Check for display images
    image_urls = []
    for item in osa_data:
        after_image = item.get('AfterImagePath', '')
        if after_image and after_image not in image_urls:
            image_urls.append(after_image)
    
    print(f"\n🖼️ Display Images Found: {len(image_urls)}")
    if image_urls:
        print(f"   First image URL: {image_urls[0][:50]}...")
        
else:
    print("❌ No data available for testing. Check your date and display ID values.")


🔍 Fetching OSA images for 2025-09-08 - ACH194
https://tamimi.impulseglobal.net//ExternalServices/IR_API.asmx/getOSAImage?Date=2025-09-08&DisplayID=ACH194
✅ Successfully fetched 50 items

📦 SKU Analysis:
   Total SKUs: 46
   Ground Truth SKUs (OSA=1): 40

📋 Sample SKUs:
   1. BAJA  EXTRA MIX 10/120G ✅
   2. BAJA  EXTRA MIX 6/280G ✅
   3. BAJA BARBEQUE MIXED NUTS 6/450G ✅
   4. BAJA DELUX MIXED NUTS 6/450G ✅
   5. BAJA DELUXE MIX BBQ 10/120G ✅

🖼️ Display Images Found: 1
   First image URL: https://tamimi.s3.amazonaws.com/MEImage/08-09-2025...


## 3. Testing Gemini Product Detection

Let's test the AI product detection functionality using a display image.


In [4]:
# Test Gemini Product Detection (FIXED)
# Note: The issue was in config.py - JSON braces in DETECTION_PROMPT_TEMPLATE needed to be escaped

if 'osa_data' in locals() and osa_data and 'image_urls' in locals() and image_urls:
    print("🤖 Testing Gemini Product Detection...")
    
    try:
        # Re-import the fixed config to get updated prompt template
        import importlib
        import config
        importlib.reload(config)
        from config import DETECTION_PROMPT_TEMPLATE
        
        # Initialize the detector
        detector = GeminiProductDetector()
        print("✅ Gemini detector initialized successfully")
        
        # Download the first image
        image_url = image_urls[0]
        print(f"📥 Downloading image from: {image_url[:50]}...")
        
        response = requests.get(image_url)
        response.raise_for_status()
        image = Image.open(BytesIO(response.content))
        
        print(f"🖼️ Image loaded: {image.size} pixels")
        
        # Prepare SKU items for detection (using ground truth SKUs with additional context)
        if 'ground_truth_skus' in locals() and ground_truth_skus:
            # Create rich items with context from OSA data (preserve duplicates)
            rich_items = []
            for item in osa_data:
                if item.get('SKU') in ground_truth_skus:
                    rich_items.append({
                        'SKU': item.get('SKU'),
                        'ShelfNo': item.get('ShelfNo') or item.get('shelfno') or item.get('shelfNo'),
                        'FacingTouching': item.get('FacingTouching') or item.get('facingtouching') or item.get('Facing') or item.get('facing')
                    })
            
            # Do NOT de-duplicate; keep repeated SKUs across shelves/placements
            to_detect = rich_items if rich_items else list(ground_truth_skus)
            
            # Prepare a readable preview of target names
            if to_detect and isinstance(to_detect[0], dict):
                preview_names = [it.get('SKU') for it in to_detect]
            else:
                preview_names = list(to_detect)
            
            print(f"🎯 Detecting {len(to_detect)} target SKUs...")
            print(f"   Target SKUs: {preview_names[:3]}{'...' if len(preview_names) > 3 else ''}")
            
            # Perform detection
            detection_results = detector.detect_products(image, to_detect)
            
            print(f"\n🔍 Detection Results:")
            if "error" in detection_results:
                print(f"❌ Error: {detection_results['error']}")
            else:
                predicted_skus = get_predicted_skus_from_results(detection_results)
                print(f"✅ Detected {len(predicted_skus)} SKUs")
                print(f"   Detected: {predicted_skus}")
                
                # Store results for accuracy testing
                test_detection_results = detection_results
                test_predicted_skus = predicted_skus
        else:
            print("⚠️ No ground truth SKUs available for detection")
            
    except Exception as e:
        print(f"❌ Detection failed: {str(e)}")
        import traceback
        traceback.print_exc()
        
else:
    print("⚠️ No OSA data or images available for testing detection")
    print("💡 You can also test with a sample result:")
    
    # Create sample detection result for demo
    sample_result = create_sample_detection_result()
    print(f"📋 Sample detection result: {sample_result}")
    test_detection_results = sample_result
    test_predicted_skus = get_predicted_skus_from_results(sample_result)


🤖 Testing Gemini Product Detection...
✅ Gemini detector initialized successfully
📥 Downloading image from: https://tamimi.s3.amazonaws.com/MEImage/08-09-2025...
🖼️ Image loaded: (3072, 4080) pixels
🎯 Detecting 44 target SKUs...
   Target SKUs: ['BAJA DIYAFA SEEDS 24/15G', 'BAJA DIYAFA PUMPKIN SEEDS 24/20G', 'BAJA SUNFLOWER SEEDS 10/250G']...

🔍 Detection Results:
✅ Detected 28 SKUs
   Detected: ['BAJA  EXTRA MIX 10/120G', 'BAJA BARBEQUE MIXED NUTS 6/450G', 'BAJA DELUX MIXED NUTS 6/450G', 'BAJA DELUXE MIX BBQ 10/120G', 'BAJA DELUXE MIX SALTED 10/120G', 'BAJA DELUXE MIX SALTED 6/280G', 'BAJA DIYAFA SEEDS 10/130G', 'BAJA DIYAFA SEEDS 24/15G', 'BAJA EXTRA MIXED NUTS 6/450G', 'BAJA JAPANESE CRACKER 12/130G', 'BAJA MIXED SWEETS & SALTY 6/450G', 'BAJA SALTED CASHEW 6/24/15GM', 'BAJA SALTED PEANUTS 10/160G', 'BAJA SUN FLOWER SEEDS 10/100G', 'BAJA SWEET SALTY MIX 6/280G', 'BAJA UNSALTED MIXED NUTS 6/450G', 'Baja Mixed Nuts Care 280 gm', 'Baja Mixed Nuts Care 450 gm', 'Baja Mixed Nuts Fibe 450 g

In [5]:
print(detection_results['prompt'])


You are analyzing a retail shelf/display image.

We provide a catalog of SKU items with helpful context per item:

- BAJA DIYAFA SEEDS 24/15G | ShelfNo: 1 | FacingTouching: 2
- BAJA DIYAFA PUMPKIN SEEDS 24/20G | ShelfNo: 1 | FacingTouching: 2
- BAJA SUNFLOWER SEEDS 10/250G | ShelfNo: 1 | FacingTouching: 2
- BAJA SUNFLOWER SEEDS SALTED 10/450 | ShelfNo: 1 | FacingTouching: 2
- BAJA DIYAFA SEEDS 10/130G | ShelfNo: 2 | FacingTouching: 2
- BAJA SALTED CASHEW 6/24/15GM | ShelfNo: 2 | FacingTouching: 2
- BAJA SUN FLOWER SEEDS 10/100G | ShelfNo: 2 | FacingTouching: 2
- BAJA SALTED PEANUTS 10/160G | ShelfNo: 3 | FacingTouching: 1
- BAJA SWEET SALTY MIX 6/280G | ShelfNo: 3 | FacingTouching: 2
- Baja Salted Cashew 120 gm | ShelfNo: 3 | FacingTouching: 1
- Baja Pistachio Lemon 120 gm | ShelfNo: 3 | FacingTouching: 1
- Baja Cashew BBQ 120 gm | ShelfNo: 3 | FacingTouching: 1
- Baja Cashew Chili 120 gm | ShelfNo: 3 | FacingTouching: 1
- Baja Pistachio BBQ 120 gm | ShelfNo: 3 | FacingTouching: 1
- B

## 4. Testing Accuracy Metrics Calculation

Let's test the accuracy metrics calculation with some sample data.


In [6]:
# Test accuracy metrics calculation
print("📊 Testing Accuracy Metrics Calculation")

# Use real data if available, otherwise create sample data
if 'ground_truth_skus' in locals() and 'test_predicted_skus' in locals():
    gt_skus = ground_truth_skus
    pred_skus = test_predicted_skus
    print(f"🎯 Using real data: {len(gt_skus)} ground truth, {len(pred_skus)} predicted")
else:
    # Create sample data for testing
    gt_skus = ["Product A", "Product B", "Product C", "Product D", "Product E"]
    pred_skus = ["Product A", "Product B", "Product F", "Product G"]  # Some correct, some false positives, some missed
    print(f"🎯 Using sample data: {len(gt_skus)} ground truth, {len(pred_skus)} predicted")

print(f"\n📋 Ground Truth SKUs: {gt_skus}")
print(f"🔍 Predicted SKUs: {pred_skus}")

# Calculate accuracy metrics
metrics = calculate_accuracy_metrics(gt_skus, pred_skus)

print(f"\n📊 Accuracy Metrics:")
print(f"   Overall Accuracy: {metrics['accuracy']:.2%}")
print(f"   Total Ground Truth: {metrics['total_ground_truth']}")
print(f"   Total Predicted: {metrics['total_predicted']}")
print(f"   Correctly Detected: {len(metrics['correctly_detected'])}")
print(f"   Missed SKUs: {len(metrics['missed_skus'])}")
print(f"   False Positives: {len(metrics['false_positives'])}")

print(f"\n✅ Correctly Detected SKUs: {metrics['correctly_detected']}")
print(f"❌ Missed SKUs (False Negatives): {metrics['missed_skus']}")
print(f"⚠️ False Positives: {metrics['false_positives']}")

# Test edge cases
print(f"\n🧪 Testing Edge Cases:")

# Empty ground truth
empty_gt_metrics = calculate_accuracy_metrics([], pred_skus)
print(f"   Empty ground truth accuracy: {empty_gt_metrics['accuracy']}")

# Empty predictions
empty_pred_metrics = calculate_accuracy_metrics(gt_skus, [])
print(f"   Empty predictions accuracy: {empty_pred_metrics['accuracy']:.2%}")

# Perfect match
perfect_metrics = calculate_accuracy_metrics(gt_skus, gt_skus)
print(f"   Perfect match accuracy: {perfect_metrics['accuracy']:.2%}")


📊 Testing Accuracy Metrics Calculation
🎯 Using real data: 40 ground truth, 28 predicted

📋 Ground Truth SKUs: ['BAJA  EXTRA MIX 10/120G', 'BAJA  EXTRA MIX 6/280G', 'BAJA BARBEQUE MIXED NUTS 6/450G', 'BAJA DELUX MIXED NUTS 6/450G', 'BAJA DELUXE MIX BBQ 10/120G', 'BAJA DELUXE MIX BBQ 6/280G', 'BAJA DELUXE MIX SALTED 10/120G', 'BAJA DELUXE MIX SALTED 6/280G', 'BAJA DELUXE MIX UNSALTED 6/280G', 'BAJA DIYAFA PUMPKIN SEEDS 24/20G', 'BAJA DIYAFA SEEDS 10/130G', 'BAJA DIYAFA SEEDS 24/15G', 'BAJA Deluxe Mix Unsalted 1X10X120G', 'BAJA EXTRA MIXED NUTS 6/450G', 'BAJA JAPANESE CRACKER 12/130G', 'BAJA MIXED SWEETS & SALTY 6/450G', 'BAJA SALTED CASHEW 6/24/15GM', 'BAJA SALTED PEANUTS 10/160G', 'BAJA SUN FLOWER SEEDS 10/100G', 'BAJA SUNFLOWER SEEDS 10/250G', 'BAJA SUNFLOWER SEEDS SALTED 10/450', 'BAJA SWEET SALTY MIX 6/280G', 'BAJA UNSALTED MIXED NUTS 6/450G', 'Baja Almond BBQ 120 gm', 'Baja Almond Lemon 120 gm', 'Baja Cashew BBQ 120 gm', 'Baja Cashew Chili 120 gm', 'Baja Mixed Nuts Care 280 gm', 'Ba

## 5. Testing File Validation and Utility Functions

Let's test some additional utility functions like file validation and SKU image URL retrieval.


In [7]:
# Test utility functions
print("🔧 Testing Utility Functions")

# Test SKU image URL retrieval
if 'osa_data' in locals() and osa_data and 'all_skus' in locals() and all_skus:
    print(f"\n🖼️ Testing SKU Image URL Retrieval:")
    test_sku = all_skus[0]  # Use first available SKU
    image_url = get_sku_image_url(test_sku, osa_data)
    
    print(f"   SKU: {test_sku}")
    print(f"   Image URL: {image_url[:50] + '...' if image_url and len(image_url) > 50 else image_url}")
    
    # Test with non-existent SKU
    fake_image_url = get_sku_image_url("NonExistentSKU", osa_data)
    print(f"   Non-existent SKU image URL: {fake_image_url}")
else:
    print(f"\n🖼️ SKU Image URL Retrieval: No OSA data available for testing")

# Test file validation function
print(f"\n📁 Testing File Validation:")

# Create a mock file object for testing
class MockFile:
    def __init__(self, name, size):
        self.name = name
        self.size = size

# Test valid file
valid_file = MockFile("test_image.jpg", 5 * 1024 * 1024)  # 5MB JPG file
is_valid, message = validate_uploaded_file(valid_file)
print(f"   Valid JPG file (5MB): {is_valid} - {message}")

# Test oversized file
oversized_file = MockFile("large_image.png", 15 * 1024 * 1024)  # 15MB PNG file
is_valid, message = validate_uploaded_file(oversized_file)
print(f"   Oversized PNG file (15MB): {is_valid} - {message}")

# Test invalid extension
invalid_file = MockFile("document.pdf", 2 * 1024 * 1024)  # 2MB PDF file
is_valid, message = validate_uploaded_file(invalid_file)
print(f"   Invalid PDF file (2MB): {is_valid} - {message}")

# Test None file
is_valid, message = validate_uploaded_file(None)
print(f"   No file uploaded: {is_valid} - {message}")

print(f"\n📋 Configuration Values:")
print(f"   Max file size: {MAX_FILE_SIZE_MB}MB")
print(f"   Allowed extensions: {ALLOWED_EXTENSIONS}")
print(f"   App colors: {list(COLORS.keys())}")


🔧 Testing Utility Functions

🖼️ Testing SKU Image URL Retrieval:
   SKU: DORITOS CRUNCH FH SCRMIN SIRCH 20/195G
   Image URL: https://tamimi.s3.amazonaws.com/UDrive/ClientData/...
   Non-existent SKU image URL: None

📁 Testing File Validation:
   Valid JPG file (5MB): True - File is valid
   Oversized PNG file (15MB): False - File too large. Maximum size is 10MB
   Invalid PDF file (2MB): False - Invalid file type. Allowed types: png, jpg, jpeg, webp
   No file uploaded: False - No file uploaded

📋 Configuration Values:
   Max file size: 10MB
   Allowed extensions: ['png', 'jpg', 'jpeg', 'webp']
   App colors: ['primary', 'secondary', 'background', 'text', 'accent', 'success']


## 6. Custom Testing Functions

Here are some additional helper functions you can use for custom testing scenarios.


In [8]:
def analyze_osa_data_summary(data):
    """
    Analyze and summarize OSA data structure and content
    """
    if not data:
        return "No data provided"
    
    summary = {
        "total_records": len(data),
        "unique_display_ids": len(get_unique_display_ids(data)),
        "unique_dates": len(get_unique_dates(data)),
        "unique_skus": len(get_unique_skus_from_api_data(data)),
        "ground_truth_skus": len(get_ground_truth_skus(data)),
        "sample_fields": list(data[0].keys()) if data else []
    }
    
    return summary

def test_detection_with_custom_skus(image_url, sku_list):
    """
    Test product detection with a custom list of SKUs
    """
    try:
        # Download image
        response = requests.get(image_url)
        response.raise_for_status()
        image = Image.open(BytesIO(response.content))
        
        # Initialize detector
        detector = GeminiProductDetector()
        
        # Perform detection
        results = detector.detect_products(image, sku_list)
        
        return {
            "success": True,
            "results": results,
            "predicted_skus": get_predicted_skus_from_results(results),
            "image_size": image.size
        }
    except Exception as e:
        return {
            "success": False,
            "error": str(e)
        }

def compare_detection_methods(ground_truth, predictions_list, method_names):
    """
    Compare multiple detection methods against ground truth
    """
    comparisons = []
    
    for i, (predictions, method_name) in enumerate(zip(predictions_list, method_names)):
        metrics = calculate_accuracy_metrics(ground_truth, predictions)
        comparisons.append({
            "method": method_name,
            "accuracy": metrics['accuracy'],
            "correctly_detected": len(metrics['correctly_detected']),
            "missed": len(metrics['missed_skus']),
            "false_positives": len(metrics['false_positives'])
        })
    
    return comparisons

# Example usage of custom functions
print("🛠️ Custom Testing Functions Defined")
print("Available functions:")
print("  - analyze_osa_data_summary(data): Analyze OSA data structure")
print("  - test_detection_with_custom_skus(image_url, sku_list): Test detection with custom SKUs")
print("  - compare_detection_methods(ground_truth, predictions_list, method_names): Compare multiple methods")

# Test data summary if we have OSA data
if 'osa_data' in locals() and osa_data:
    print(f"\n📊 Current OSA Data Summary:")
    summary = analyze_osa_data_summary(osa_data)
    for key, value in summary.items():
        if key == "sample_fields":
            print(f"   {key}: {value[:5]}{'...' if len(value) > 5 else ''}")
        else:
            print(f"   {key}: {value}")


🛠️ Custom Testing Functions Defined
Available functions:
  - analyze_osa_data_summary(data): Analyze OSA data structure
  - test_detection_with_custom_skus(image_url, sku_list): Test detection with custom SKUs
  - compare_detection_methods(ground_truth, predictions_list, method_names): Compare multiple methods

📊 Current OSA Data Summary:
   total_records: 29
   unique_display_ids: 1
   unique_dates: 1
   unique_skus: 27
   ground_truth_skus: 24
   sample_fields: ['DOEntry', 'DisplayID', 'ArticleNo', 'Upccode', 'SKU']...


## 7. Example Usage Scenarios

Here are some example scenarios showing how to use the functions for different testing purposes.


In [9]:
# Example Scenario 1: Quick API Data Exploration
print("📖 Example Scenario 1: Quick API Data Exploration")
print("="*50)

# Example code to explore what's available in the API
def quick_api_exploration():
    data, error = fetch_all_osa_data()
    if error:
        print(f"Error: {error}")
        return
    
    display_ids = get_unique_display_ids(data)
    dates = get_unique_dates(data)
    
    print(f"Available Display IDs: {display_ids[:3]}...")
    print(f"Latest dates: {[d['display'] for d in dates[:3]]}")
    
    # Pick first available combination for testing
    if display_ids and dates:
        test_data, _ = fetch_osa_images(dates[0]['date'], display_ids[0])
        if test_data:
            skus = get_unique_skus_from_api_data(test_data)
            gt_skus = get_ground_truth_skus(test_data)
            print(f"Found {len(skus)} total SKUs, {len(gt_skus)} with OSA=1")

print("Function: quick_api_exploration()")
print("Usage: Call this function to quickly explore available data")

print("\n" + "="*50)

# Example Scenario 2: Testing Detection with Custom Image
print("📖 Example Scenario 2: Testing Detection with Custom Image")
print("="*50)

# Example code for testing with a custom image URL and SKU list
def test_custom_detection():
    # Example with a hypothetical image URL and SKU list
    image_url = "https://example.com/shelf_image.jpg"  # Replace with actual URL
    custom_skus = ["Coca Cola", "Pepsi", "Water Bottle", "Chips"]
    
    result = test_detection_with_custom_skus(image_url, custom_skus)
    
    if result["success"]:
        print(f"Detection successful!")
        print(f"Image size: {result['image_size']}")
        print(f"Predicted SKUs: {result['predicted_skus']}")
        
        # Calculate accuracy if you know ground truth
        known_ground_truth = ["Coca Cola", "Water Bottle"]  # What's actually in the image
        metrics = calculate_accuracy_metrics(known_ground_truth, result['predicted_skus'])
        print(f"Accuracy: {metrics['accuracy']:.2%}")
    else:
        print(f"Detection failed: {result['error']}")

print("Function: test_custom_detection()")
print("Usage: Modify image_url and custom_skus, then call the function")

print("\n" + "="*50)

# Example Scenario 3: Comparing Multiple Detection Approaches
print("📖 Example Scenario 3: Comparing Multiple Detection Approaches")
print("="*50)

# Example code for comparing different detection methods
def compare_approaches():
    ground_truth = ["Product A", "Product B", "Product C"]
    
    # Simulate different detection method results
    method1_results = ["Product A", "Product B"]  # Conservative approach
    method2_results = ["Product A", "Product B", "Product C", "Product D"]  # Aggressive approach
    method3_results = ["Product A", "Product C", "Product E"]  # Mixed results
    
    predictions = [method1_results, method2_results, method3_results]
    method_names = ["Conservative", "Aggressive", "Mixed"]
    
    comparisons = compare_detection_methods(ground_truth, predictions, method_names)
    
    print("Method Comparison:")
    for comp in comparisons:
        print(f"  {comp['method']:12} | Accuracy: {comp['accuracy']:.2%} | "
              f"Correct: {comp['correctly_detected']} | "
              f"Missed: {comp['missed']} | False+: {comp['false_positives']}")

print("Function: compare_approaches()")
print("Usage: Call this function to see method comparison example")

print("\n" + "="*70)
print("💡 Tips for Using This Notebook:")
print("1. Run cells sequentially to build up the data and test functions")
print("2. Modify the test_date and test_display_id in cell 5 based on available data")
print("3. Check that GOOGLE_API_KEY is properly configured for AI detection")
print("4. Use the custom functions for your own testing scenarios")
print("5. Save any interesting results using json.dumps() or pandas DataFrames")


📖 Example Scenario 1: Quick API Data Exploration
Function: quick_api_exploration()
Usage: Call this function to quickly explore available data

📖 Example Scenario 2: Testing Detection with Custom Image
Function: test_custom_detection()
Usage: Modify image_url and custom_skus, then call the function

📖 Example Scenario 3: Comparing Multiple Detection Approaches
Function: compare_approaches()
Usage: Call this function to see method comparison example

💡 Tips for Using This Notebook:
1. Run cells sequentially to build up the data and test functions
2. Modify the test_date and test_display_id in cell 5 based on available data
3. Check that GOOGLE_API_KEY is properly configured for AI detection
4. Use the custom functions for your own testing scenarios
5. Save any interesting results using json.dumps() or pandas DataFrames
